# Azure Evaluation On Stuttering

1. Turn original transcripts into dataframes
2. Read generated transcript dataframes
3. Get Error & Stuttering count
4. Get Error & Stuttering Type ==> show which Stuttering type is more prone to error
5. Calculate Correlation Score
6. Generate Heatmap
7. Summarize Trend

*** 

### Error Metrics
1. Net / Total Word Error Rate
2. Word Error Rate Specific After Cleaning other Stuttering Type Annotations

***

## Using Custom Kernel on SCC

SCC sometimes has the problem with installed library not importable [`module not found` error], this is an alternative.

Assuming you have a conda environment created, you would do the following:
1. `conda install -c anaconda ipykernel` 
2. `python -m ipykernel install --user --name=<env name>`
3. If the new kernel cannot be found, relaunch a new SCC instance

**Remember to switch to the conda env kernel**

In [ ]:
ds_transcript_path = "/projectnb/ds549/projects/AImpower/datasets/updated_annotation_deid_full"

In [ ]:
!pip install pandas numpy scipy tqdm

---

## Word Error Rate

Objectives:
* split sequence into characters
* count:
    * deletion: missing words
    * substitutions: wrongly recognized words
    * insertions: extra words

In [ ]:
def wer(candidate, reference):
    """
    Parameter(s)
    ------------
    candidate ==> generated transcript
    reference ==> dataset transcript
    """
    
    candidate_tokens = list(candidate)
    reference_tokens = list(reference)
    
    cand_len = len(candidate_tokens)
    ref_len = len(reference_tokens)
    
    dist_mat = np.zeros((ref_len, cand_len), dtype=int)
    
    for i in range(ref_len):
        dist_mat[i][0] = i
    for j in range(cand_len):
        dist_mat[0][j] = j
        
    for i in range(1, ref_len):
        for j in range(1, cand_len):
            if (candidate_tokens[j - 1] == reference_tokens[i - 1]):
                cost = 0
            else:
                cost = 1
                
            dist_mat[i][j] = min(
                dist_mat[i-1][j] + 1,
                dist_mat[i][j-1] + 1,
                dist_mat[i-1][j-1] + cost
            )
            # print(dist_mat)
            
    wer = dist_mat[-1][-1] / len(reference_tokens)
    return wer

***

## Imports and Data Ingestion

In [ ]:
import pandas as pd
import numpy as np
import scipy
import os
from tqdm import tqdm
import re
import sys

In [ ]:
net_data = pd.DataFrame(columns=["Filename", "Start_time", "End_time", "Transcript"]) 
net_aigenerated_data_azure = pd.read_csv("/projectnb/ds549/projects/AImpower/datasets/generated-transcripts/Azure.csv", delimiter=",")

del net_aigenerated_data_azure[net_aigenerated_data_azure.columns[0]]

In [ ]:
for folder in os.listdir(ds_transcript_path):
    if folder == "command_stats.xlsx" or folder == "command_stats.csv":
        continue
    for audio_sample in os.listdir(os.path.join(ds_transcript_path, f"{folder}")):
        if ("_A.txt" in audio_sample):
            net_data = pd.concat([net_data, pd.read_csv(os.path.join(ds_transcript_path, f"{folder}/{audio_sample}"), sep="\t", names=["Start_time", "End_time", "Transcript"]).assign(Filename=f"D{folder}_A")])
        if ("_B.txt" in audio_sample):
            net_data = pd.concat([net_data, pd.read_csv(os.path.join(ds_transcript_path, f"{folder}/{audio_sample}"), sep="\t", names=["Start_time", "End_time", "Transcript"]).assign(Filename=f"D{folder}_B")])
        if ("P" in audio_sample):
            net_data = pd.concat([net_data, pd.read_csv(os.path.join(ds_transcript_path, f"{folder}/{audio_sample}"), sep="\t", names=["Start_time", "End_time", "Transcript"]).assign(Filename=f"P{folder}")])

In [ ]:
mask_pattern = r"\<.*?\>"
repetition_pattern = r"\[.*?\]"
annotation_pattern = r"/\w"


net_data = net_data.assign(Cleaned_Transcript=net_data['Transcript'].apply(lambda x: re.sub(annotation_pattern, "", re.sub(repetition_pattern, "", re.sub(mask_pattern, "", x)))))
net_data = net_data.assign(Stutterance_Count=net_data['Transcript'].apply(lambda x: len(re.findall(mask_pattern, x)) + len(re.findall(repetition_pattern, x)) + len(re.findall(annotation_pattern, x))))

In [ ]:
net_data

In [ ]:
net_aigenerated_data_azure

**Now we have raw data of all audio transcriptions from datasets [updated_annotation_deid_full] in `net_data` and AI predicted transcriptions in `net_aigenerated_data_azure`**

***

## WER Analysis

In [ ]:
na_count_large = 0
na_count_cleaned = 0
for index, row in tqdm(net_aigenerated_data_azure.iterrows(), total=len(net_aigenerated_data_azure)):
    
    mask_large = (
        (net_aigenerated_data_azure["Filename"] == row["Filename"]) &
        (net_aigenerated_data_azure["Start_time"] == row["Start_time"])
    )

    mask_net = (
        (net_data["Filename"] == row["Filename"]) &
        (net_data["Start_time"] == row["Start_time"])
    )

    
    large_row = net_aigenerated_data_azure.loc[mask_large]
    net_row = net_data.loc[mask_net]

    # print(large_row)
    # print('\n\n\n\n')
    # print(net_row)
    
    if large_row.empty or net_row.empty:
        print("Skipping: One of the rows is empty.")
        continue
        
    azure = large_row["Azure"].values[0]
    cleaned_transcript = net_row["Cleaned_Transcript"].values[0]
    
    if pd.isna(azure) or not isinstance(azure, str):
        print("Skipping due to missing or non-string Azure.")
        na_count_large = na_count_large + 1
        continue
    if pd.isna(cleaned_transcript) or not isinstance(cleaned_transcript, str):
        print("Skipping due to missing or non-string Cleaned_Transcript.")
        na_count_cleaned = na_count_cleaned + 1
        continue

    try:
        
        wer_value = wer(azure, cleaned_transcript)
        
        net_aigenerated_data_azure.loc[mask_large, "WER"] = wer_value
        
        stutterance_count = net_row["Stutterance_Count"].values[0]
        net_aigenerated_data_azure.loc[mask_large, "Stutterance_Count"] = stutterance_count

        # Verify assignment
        # print(f'Assigned Stutterance_Count: {stutterance_count}')
        # print(net_aigenerated_data_azure.loc[mask_large, "Stutterance_Count"])

    except Exception as e:
        print(f'ERROR: {e}')
        print('Occurred with the following data:')
        print(large_row)
        print(net_row)
        
net_aigenerated_data_azure = net_aigenerated_data_azure.assign(NA_Count=na_count_large)
net_aigenerated_data_azure = net_aigenerated_data_azure.assign(NA_Cleaned_Count=na_count_cleaned)

In [ ]:
net_aigenerated_data_azure

In [ ]:
net_aigenerated_data_azure.to_csv('net_aigenerated_data_azure_performance_stu.csv', sep=',')

***

## Visualization of Relationship between Stuttering Count and Word Error Rate

In [ ]:
## Load data from csv if starting here

net_aigenerated_data_azure = pd.read_csv('/projectnb/ds549/projects/AImpower/evaluation-azure/net_aigenerated_data_azure_performance_stu.csv', delimiter=',')

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
## Null value plots

nonnull_count_large = net_aigenerated_data_azure["NA_Count"].count() - net_aigenerated_data_azure.iloc[0]["NA_Count"]
null_count_large = net_aigenerated_data_azure.iloc[0]["NA_Count"]

nonnull_count_cleaned = net_aigenerated_data_azure["NA_Cleaned_Count"].count() - net_aigenerated_data_azure.iloc[0]["NA_Cleaned_Count"]
null_count_cleaned = net_aigenerated_data_azure.iloc[0]["NA_Cleaned_Count"]


data = {
    "NA Values": [null_count_large, null_count_cleaned],
    "Non NA Values": [nonnull_count_large, nonnull_count_cleaned],
}

species = (
    "Azure",
    "Cleaned Ground Truth"
)

width = 0.5

fig, ax = plt.subplots()
bottom = np.zeros(2)

for na, count in data.items():
    p = ax.bar(species, count, width, label=na, bottom=bottom)
    bottom += count


ax.set_title("NA value counting", fontsize=16)
ax.set_xlabel("Source", fontsize=14)
ax.set_ylabel("Count", fontsize=14)
ax.legend()
ax.grid(True)

plt.show()

In [ ]:
plt.figure(figsize=(6, 4))
plt.scatter(
    net_aigenerated_data_azure["Stutterance_Count"], 
    net_aigenerated_data_azure["WER"], 
    alpha=0.7  # Handle overlapping points
)

plt.title("WER vs Stutterance Count", fontsize=16)
plt.xlabel("Stutterance Count", fontsize=14)
plt.ylabel("WER", fontsize=14)
plt.grid(True)
plt.show()

In [ ]:
import seaborn as sns
from matplotlib.colors import LinearSegmentedColormap

net_aigenerated_data_azure['WER_Binned'] = np.round(net_aigenerated_data_azure['WER'], 2)

grouped_data = net_aigenerated_data_azure.groupby(
    ['Stutterance_Count', 'WER_Binned']
).size().reset_index(name='Count')
heatmap_data = grouped_data.pivot(index='WER_Binned', columns='Stutterance_Count', values='Count').fillna(0)

plt.figure(figsize=(6, 4))
sns.heatmap(
    heatmap_data, cmap='cool', annot=False, fmt='g', cbar=True
)

plt.title("Stutterance Count vs WER (Color = Number of Cases)", fontsize=16)
plt.xlabel("Stutterance Count", fontsize=14)
plt.ylabel("WER (Binned)", fontsize=14)

plt.show()

In [ ]:
from scipy.stats import spearmanr
rho, p = spearmanr(net_aigenerated_data_azure.dropna()['Stutterance_Count'], net_aigenerated_data_azure.dropna()['WER'])
print(f"p-value = {p}")
print(f"rho = {rho}")

***

## ROUGE-N/L Scores (Semantic Evaluation)

In [ ]:
from rouge_chinese import Rouge
import jieba

In [ ]:
rouge = Rouge()

In [ ]:
print_ = True

for index, row in tqdm(net_aigenerated_data_azure.iterrows(), total=len(net_aigenerated_data_azure)):
    
    mask_large = (
        (net_aigenerated_data_azure["Filename"] == row["Filename"]) &
        (net_aigenerated_data_azure["Start_time"] == row["Start_time"])
    )

    mask_net = (
        (net_data["Filename"] == row["Filename"]) &
        (net_data["Start_time"] == row["Start_time"])
    )

    
    large_row = net_aigenerated_data_azure.loc[mask_large]
    net_row = net_data.loc[mask_net]

    # print(large_row)
    # print('\n\n\n\n')
    # print(net_row)
    
    if large_row.empty or net_row.empty:
        print("Skipping: One of the rows is empty.")
        continue
        
    azure = large_row["Azure"].values[0]
    cleaned_transcript = net_row["Cleaned_Transcript"].values[0]
    
    if pd.isna(azure) or not isinstance(azure, str):
        print("Skipping due to missing or non-string Azure.")
        continue
    if pd.isna(cleaned_transcript) or not isinstance(cleaned_transcript, str):
        print("Skipping due to missing or non-string Cleaned_Transcript.")
        continue

    try:
        
        scores = rouge.get_scores(' '.join(jieba.cut(azure)), ' '.join(jieba.cut(cleaned_transcript)))
        
        net_aigenerated_data_azure.loc[mask_large, "rouge1-precision"] = scores[0]["rouge-1"]["p"]
        net_aigenerated_data_azure.loc[mask_large, "rouge1-recall"] = scores[0]["rouge-1"]["r"]
        net_aigenerated_data_azure.loc[mask_large, "rouge1-f1"] = scores[0]["rouge-1"]["f"]
        
        
        net_aigenerated_data_azure.loc[mask_large, "rouge2-precision"] = scores[0]["rouge-2"]["p"]
        net_aigenerated_data_azure.loc[mask_large, "rouge2-recall"] = scores[0]["rouge-2"]["r"]
        net_aigenerated_data_azure.loc[mask_large, "rouge2-f1"] = scores[0]["rouge-2"]["f"]
        
        
        net_aigenerated_data_azure.loc[mask_large, "rougel-precision"] = scores[0]["rouge-l"]["p"]
        net_aigenerated_data_azure.loc[mask_large, "rougel-recall"] = scores[0]["rouge-l"]["r"]
        net_aigenerated_data_azure.loc[mask_large, "rougel-f1"] = scores[0]["rouge-l"]["f"]
        
        stutterance_count = net_row["Stutterance_Count"].values[0]
        net_aigenerated_data_azure.loc[mask_large, "Stutterance_Count"] = stutterance_count

        if (print_):
            print(net_aigenerated_data_azure)
            print_ = False
        
        # Verify assignment
        # print(f'Assigned Stutterance_Count: {stutterance_count}')
        # print(net_aigenerated_data_azure.loc[mask_large, "Stutterance_Count"])

    except Exception as e:
        print(f'ERROR: {e}')
        print('Occurred with the following data:')
        print(large_row)
        print(net_row)

In [ ]:
net_aigenerated_data_azure

In [ ]:
net_aigenerated_data_azure.to_csv('net_aigenerated_data_azure_performance_stu.csv', sep=',')

***

## Visualization of Relationship between Stuttering Count and Rouge Scores

In [ ]:
## Load data from csv if starting here

net_aigenerated_data_azure = pd.read_csv('/projectnb/ds549/projects/AImpower/evaluation-azure/net_aigenerated_data_azure_performance_stu.csv', delimiter=',')

In [ ]:
plt.figure(figsize=(6, 4))
plt.scatter(
    net_aigenerated_data_azure["Stutterance_Count"], 
    net_aigenerated_data_azure["rouge1-precision"], 
    facecolors="none", edgecolors='r',
    marker="8",
    alpha=0.7  # Handle overlapping points
)

plt.scatter(
    net_aigenerated_data_azure["Stutterance_Count"], 
    net_aigenerated_data_azure["rouge1-recall"], 
    facecolors="none", edgecolors='g',
    marker="^",
    alpha=0.7  # Handle overlapping points
)

plt.scatter(
    net_aigenerated_data_azure["Stutterance_Count"], 
    net_aigenerated_data_azure["rouge1-f1"], 
    facecolors="none", edgecolors='b',
    marker=".",
    alpha=0.7  # Handle overlapping points
)

plt.title("Rouge-1 vs Stutterance Count", fontsize=16)
plt.xlabel("Stutterance Count", fontsize=14)
plt.ylabel("Rouge Score", fontsize=14)
plt.grid(True)
plt.show()

In [ ]:
plt.figure(figsize=(6, 4))
plt.scatter(
    net_aigenerated_data_azure["Stutterance_Count"], 
    net_aigenerated_data_azure["rouge2-precision"], 
    facecolors="none", edgecolors='r',
    marker="8",
    alpha=0.7  # Handle overlapping points
)

plt.scatter(
    net_aigenerated_data_azure["Stutterance_Count"], 
    net_aigenerated_data_azure["rouge2-recall"], 
    facecolors="none", edgecolors='g',
    marker="^",
    alpha=0.7  # Handle overlapping points
)

plt.scatter(
    net_aigenerated_data_azure["Stutterance_Count"], 
    net_aigenerated_data_azure["rouge2-f1"], 
    facecolors="none", edgecolors='b',
    marker=".",
    alpha=0.7  # Handle overlapping points
)

plt.title("Rouge-2 vs Stutterance Count", fontsize=16)
plt.xlabel("Stutterance Count", fontsize=14)
plt.ylabel("Rouge Score", fontsize=14)
plt.grid(True)
plt.show()

In [ ]:
plt.figure(figsize=(6, 4))
plt.scatter(
    net_aigenerated_data_azure["Stutterance_Count"], 
    net_aigenerated_data_azure["rougel-precision"], 
    facecolors="none", edgecolors='r',
    marker="8",
    alpha=0.7  # Handle overlapping points
)

plt.scatter(
    net_aigenerated_data_azure["Stutterance_Count"], 
    net_aigenerated_data_azure["rougel-recall"], 
    facecolors="none", edgecolors='g',
    marker="^",
    alpha=0.7  # Handle overlapping points
)

plt.scatter(
    net_aigenerated_data_azure["Stutterance_Count"], 
    net_aigenerated_data_azure["rougel-f1"], 
    facecolors="none", edgecolors='b',
    marker=".",
    alpha=0.7  # Handle overlapping points
)

plt.title("Rouge-L vs Stutterance Count", fontsize=16)
plt.xlabel("Stutterance Count", fontsize=14)
plt.ylabel("Rouge Score", fontsize=14)
plt.grid(True)
plt.show()

## Correlations between Stuttering and Rouge Scores

In [ ]:
from scipy.stats import spearmanr
rho, p = spearmanr(net_aigenerated_data_azure.dropna()['Stutterance_Count'], net_aigenerated_data_azure.dropna()['rouge1-precision'])
print(f"p-value [stuttering count & rouge-1 precision] = {p}")
print(f"rho [stuttering count & rouge-1 precision] = {rho}")

rho, p = spearmanr(net_aigenerated_data_azure.dropna()['Stutterance_Count'], net_aigenerated_data_azure.dropna()['rouge1-recall'])
print(f"p-value [stuttering count & rouge-1 recall] = {p}")
print(f"rho [stuttering count & rouge-1 recall] = {rho}")

rho, p = spearmanr(net_aigenerated_data_azure.dropna()['Stutterance_Count'], net_aigenerated_data_azure.dropna()['rouge1-f1'])
print(f"p-value [stuttering count & rouge-1 f1] = {p}")
print(f"rho [stuttering count & rouge-1 f1] = {rho}")